In [1]:
#!/usr/bin/env python3

import rospy
import time
import jetson.inference
import jetson.utils
import cv2, numpy

import argparse
import sys
import os
from std_msgs.msg import String, Int64
import ipywidgets

parser = argparse.ArgumentParser(description="Locate objects in a live camera stream using an object detection DNN.", 
                                 formatter_class=argparse.RawTextHelpFormatter, epilog=jetson.inference.detectNet.Usage() +
                                 jetson.utils.videoSource.Usage() + jetson.utils.videoOutput.Usage() + jetson.utils.logUsage())

parser.add_argument("input_URI", type=str, default="", nargs='?', help="URI of the input stream")
parser.add_argument("output_URI", type=str, default="", nargs='?', help="URI of the output stream")
parser.add_argument("--network", type=str, default="ssd-mobilenet-v2", help="pre-trained model to load (see below for options)")
parser.add_argument("--overlay", type=str, default="box,labels,conf", help="detection overlay flags (e.g. --overlay=box,labels,conf)\nvalid combinations are:  'box', 'labels', 'conf', 'none'")
parser.add_argument("--threshold", type=float, default=0.5, help="minimum detection threshold to use") 

is_headless = ["--headless"] if sys.argv[0].find('console.py') != -1 else [""]

try:
	opt = parser.parse_known_args()[0]
except:
	print("")
	parser.print_help()
	sys.exit(0)

net = jetson.inference.detectNet(argv=['--model=models/ssd-mobilenet.onnx', '--labels=models/labels.txt', '--input-blob=input_0', '--output-cvg=scores', '--output-bbox=boxes'])
rospy.init_node('detect_control')

pub = rospy.Publisher('/jetbot_motors/cmd_str', String, queue_size=3)
pub2 = rospy.Publisher('cx', Int64, queue_size = 1)
pub3 = rospy.Publisher('/alarm', String, queue_size=1)

camera = jetson.utils.videoSource(argv=["csi://0", "--input-flip=horizontal"])
output = jetson.utils.videoOutput("display://0")

In [2]:
from IPython.display import display
import ipywidgets

def pre(img):
	image = jetson.utils.cudaToNumpy(img)
	image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
	image = bytes(cv2.imencode('.jpg', image[:, ::-1, :])[1])
	return image

image_w = ipywidgets.Image(format='jpeg')
display(image_w)

Image(value=b'', format='jpeg')

In [3]:
image_f = ipywidgets.Image(format='jpeg')
display(image_f)

Image(value=b'', format='jpeg')

In [4]:
image_co = ipywidgets.Image(format='jpeg')
display(image_co)

Image(value=b'', format='jpeg')

In [5]:
def line_following(frame):
    frame = jetson.utils.cudaToNumpy(frame)
    f_crop = frame[400:720, :]
    f_crop = cv2.cvtColor(f_crop, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(f_crop, cv2.COLOR_BGR2HSV)
    global cx
    global cy
    image_co.value = bytes(cv2.imencode('.jpg', f_crop[:, ::-1, :])[1])
    lower = numpy.array([22, 93, 0], dtype="uint8")
    upper = numpy.array([45, 255, 255], dtype="uint8")
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(f_crop, f_crop, mask=mask)
    
    M = cv2.moments(mask)
    if M['m00'] > 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
        cv2.line(result, (cx, 0), (cx, 720), (255, 0, 0), 2)
        cv2.line(result, (0, cy), (1280, cy), (255, 0, 0), 2)
        pub2.publish(cx)
    image_f.value = bytes(cv2.imencode('.jpg', result[:, ::-1, :])[1])
    cv2.imshow('frame', result)

In [ ]:
while True:
	img = camera.Capture()
	detections = net.Detect(img, overlay='NONE')
    
	num = len(detections)
	if num > 0:
		for detection in detections:
			class_name = net.GetClassDesc(detection.ClassID)
			class_conf = 100*detection.Confidence
			class_box = int(detection.Area)
			if class_name == "STOP" and class_box >= 30000:
				pub.publish("stop")
				rospy.loginfo("Stop sign is detected")
			elif class_name == "LEFT" and class_box >= 30000:
				pub.publish("left")
				rospy.loginfo("Left sign is detected")
			elif class_name == "RIGHT"and class_box >= 20000:
				pub.publish("right")
				rospy.loginfo("Right sign is detected")
			elif class_name == "SLOW" or class_name == "CHILD" and class_box >= 50000 and class_conf >= 90:
				pub.publish("slow")
				rospy.loginfo("school zone")
				rospy.sleep(3)
			elif class_name == "POTHOLE" and class_conf >= 80:
				pub3.publish("pothole")
				rospy.loginfo("Pothole is detected")
				line_following(img)
			elif class_name == "CRACK" and class_conf >= 60:
				pub3.publish("crack")
				rospy.loginfo("Crack is detected")
				pub.publish("forward")
			else:
				rospy.loginfo("Confidence: %d" , int(class_conf))
				line_following(img)
	else:
		line_following(img)
		rospy.loginfo("Nothing is detected")
        
	output.Render(img)
	image_w.value = pre(img)
	output.SetStatus("{:s} | Network {:.0f} FPS".format(opt.network, net.GetNetworkFPS()))
	net.PrintProfilerTimes()

	if not camera.IsStreaming() or not output.IsStreaming():
		break
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break


[INFO] [1624187239.349793]: Nothing is detected
[INFO] [1624187240.316590]: Nothing is detected
[INFO] [1624187240.911672]: Nothing is detected
[INFO] [1624187241.338650]: Nothing is detected
[INFO] [1624187241.607861]: Nothing is detected
[INFO] [1624187241.870010]: Nothing is detected
[INFO] [1624187242.183970]: Nothing is detected
[INFO] [1624187242.699092]: Nothing is detected
[INFO] [1624187242.982726]: Nothing is detected
[INFO] [1624187243.262166]: Nothing is detected
[INFO] [1624187243.508807]: Nothing is detected
[INFO] [1624187243.756310]: Nothing is detected
[INFO] [1624187244.128896]: Nothing is detected
[INFO] [1624187244.442837]: Nothing is detected
[INFO] [1624187244.721899]: Nothing is detected
[INFO] [1624187245.013658]: Nothing is detected
[INFO] [1624187245.276468]: Nothing is detected
[INFO] [1624187245.556865]: Nothing is detected
[INFO] [1624187245.908715]: Nothing is detected
[INFO] [1624187246.208875]: Nothing is detected
[INFO] [1624187246.485058]: Nothing is d